In [1]:
import phrase_entity_extraction as ph

In [2]:
sent = """APT 10 270 WEBERS LANE""".lower() 

max_address = ph.return_max_address4(ph.seq, sent)
if sent.upper().strip() == max_address.upper().strip():
    print("ALL GOOD!")
else:
    print()
    print("## ************** NOT THE SAME!!!! *********************##\n")
encoding = ph.encode_from_word_list(ph.w(sent))
print("Max Address: ", max_address, "\n\nWORD ARRAY: ", ph.w(sent), "\n\nENCODING: ", encoding)
print('\n\n{},"{}"'.format(sent.upper(), encoding))


## ************** NOT THE SAME!!!! *********************##

Max Address:  10 270 webers lane 

WORD ARRAY:  ['apt', '10', '270', 'webers', 'lane'] 

ENCODING:  [['APT'], ['DIGIT', 'ADR_HEAD'], ['DIGIT', 'ADR_HEAD'], ['ALPHA'], ['WORDWAY']]


APT 10 270 WEBERS LANE,"[['APT'], ['DIGIT', 'ADR_HEAD'], ['DIGIT', 'ADR_HEAD'], ['ALPHA'], ['WORDWAY']]"


In [3]:
ph.return_best_fit(ph.seq, sent)

[[0, 2, 2, ['DIR'], 'apt 10'], [2, 5, 3, ['DIR'], '270 webers lane']]

In [4]:
ph.seq.look_ahead(ph.encode_from_word_list(ph.w(sent.lower()))).get_active_sequences()

[]

In [5]:
ph.seq.look_ahead(ph.encode_from_word_list(ph.w(sent.lower()))).get_next_values()

[]

In [6]:
arr_cands = ph.get_markers(ph.seq, sent, ['_ADDRESS_', '_POBOX_', '_SUITE_', '_DIR_'])
arr_cands

[[0, 2, 2, ['_SUITE_'], 'apt 10'],
 [0, 3, 3, ['_SUITE_'], 'apt 10 270'],
 [1, 2, 1, ['_SUITE_'], '10'],
 [1, 5, 4, ['_ADDRESS_'], '10 270 webers lane'],
 [2, 3, 1, ['_SUITE_'], '270'],
 [2, 4, 2, ['_ADDRESS_'], '270 webers'],
 [2, 5, 3, ['_ADDRESS_'], '270 webers lane']]

In [7]:
ph.decompose_into_dictionary_words(ph.w(sent), ph.seq, ['_ADDRESS_', '_SUITE_', '_POBOX_'])

([-1, 2, 3, 2, 4],
 ['', ['_SUITE_'], ['_SUITE_'], ['_ADDRESS_'], ['_ADDRESS_']],
 [['_SUITE_']],
 [(['_SUITE_'], 'apt 10 270')])

In [8]:
filtered_markers = [marker for marker in arr_cands if not (marker[0] == 0 and marker[2] == 1)]
print(filtered_markers)
arr_cands

[[0, 2, 2, ['_SUITE_'], 'apt 10'], [0, 3, 3, ['_SUITE_'], 'apt 10 270'], [1, 2, 1, ['_SUITE_'], '10'], [1, 5, 4, ['_ADDRESS_'], '10 270 webers lane'], [2, 3, 1, ['_SUITE_'], '270'], [2, 4, 2, ['_ADDRESS_'], '270 webers'], [2, 5, 3, ['_ADDRESS_'], '270 webers lane']]


[[0, 2, 2, ['_SUITE_'], 'apt 10'],
 [0, 3, 3, ['_SUITE_'], 'apt 10 270'],
 [1, 2, 1, ['_SUITE_'], '10'],
 [1, 5, 4, ['_ADDRESS_'], '10 270 webers lane'],
 [2, 3, 1, ['_SUITE_'], '270'],
 [2, 4, 2, ['_ADDRESS_'], '270 webers'],
 [2, 5, 3, ['_ADDRESS_'], '270 webers lane']]

In [9]:
def entitys_overlap(ent1, ent2):
    if ent1[1] <= ent2[0] or ent2[1] <= ent1[0]:
        return False
    else:
        return True

def get_sorted_entity(_markers, entity):
    entities = [arr for arr in _markers if arr[3][0] == entity]
    entities.sort(key=lambda x: int(x[2]))
    return entities
def get_all_sorted_entity(_markers, axis):
    """axis 0:left, 1:right, 2:length, 3:lst<type>, 4:str"""
    markers.sort(key=lambda x: int(x[axis]))
    return markers

In [10]:
def get_markers(seq, sent, lst_targets):
    """Input is like '123 main str' and returns a list of lists
        RETURNS: a list of list, each list being a candidate and having these values.
            idx_beg, idx_end, length, matches (ADDRESS etc), sequence
        ATTN!!  this lowercases stuff, TODO: Generalize this so it doesn't need lowercasing
    """
    sent = str(sent).lower().strip()
    arr_w = ph.w(sent)
    idx_tail = len(arr_w)
    markers = []

    for idx_beg in range(idx_tail):
        for idx_end in range(idx_beg + 1, idx_tail +1):
            next_values = seq.look_ahead(ph.encode_from_word_list(arr_w[idx_beg:idx_end])).get_next_values()
            matches = list(set(next_values) & set(lst_targets) )
            if matches:
                markers.append([idx_beg, idx_end, idx_end - idx_beg, matches, ' '.join(arr_w[idx_beg:idx_end])])

    return markers


st = sent
markers = get_markers(ph.seq, st, ['_ADDRESS_', '_POBOX_', '_SUITE_', '_DIR_'])
markers

[[0, 2, 2, ['_SUITE_'], 'apt 10'],
 [0, 3, 3, ['_SUITE_'], 'apt 10 270'],
 [1, 2, 1, ['_SUITE_'], '10'],
 [1, 5, 4, ['_ADDRESS_'], '10 270 webers lane'],
 [2, 3, 1, ['_SUITE_'], '270'],
 [2, 4, 2, ['_ADDRESS_'], '270 webers'],
 [2, 5, 3, ['_ADDRESS_'], '270 webers lane']]

In [11]:
def sift_markers(markers):
    boxes = [marker for marker in markers if marker[3][0] == '_POBOX_']
    addresses = [marker for marker in markers if marker[3][0] == '_ADDRESS_']
    suites = [marker for marker in markers if marker[3][0] == '_SUITE_']
    dirs = [marker for marker in markers if marker[3][0] == '_DIR_']
    new_markers = boxes[:]
#     for address in addresses:
#         if not any([entitys_overlap(address, item) for item in new_markers]):
#             new_markers.append(address)
    new_markers.extend(addresses)
    for suite in suites:
        if not any([entitys_overlap(suite, item) for item in new_markers]):
            new_markers.append(suite)
    for diritem in dirs:
        if not any([entitys_overlap(diritem, item) for item in new_markers]):
            new_markers.append(diritem)
    return new_markers

sift_markers(markers)

[[1, 5, 4, ['_ADDRESS_'], '10 270 webers lane'],
 [2, 4, 2, ['_ADDRESS_'], '270 webers'],
 [2, 5, 3, ['_ADDRESS_'], '270 webers lane']]

In [12]:


def get_best_fit_4(seq, st, lst_entities):
    assert isinstance(lst_entities,list), "expects lst_entities is [ str, str ]"
    assert isinstance(lst_entities[0], str), "expects lst_entities is [ str, str ]"
    assert isinstance(st, str), "expects st to be a str"
    def sift_markers(markers):
        boxes = [marker for marker in markers if marker[3][0] == '_POBOX_']
        addresses = [marker for marker in markers if marker[3][0] == '_ADDRESS_']
        suites = [marker for marker in markers if marker[3][0] == '_SUITE_']
        dirs = [marker for marker in markers if marker[3][0] == '_DIR_']
        new_markers = boxes[:]
#         for address in addresses:
#             if not any([entitys_overlap(address, item) for item in new_markers]):
#                 new_markers.append(address)
        new_markers.extend(addresses)
        for suite in suites:
            if not any([entitys_overlap(suite, item) for item in new_markers]):
                new_markers.append(suite)
        for diritem in dirs:
            if not any([entitys_overlap(diritem, item) for item in new_markers]):
                new_markers.append(diritem)
        return new_markers
    def get_sorted_indexes(markers, which, up=True):
        sorted_list = sorted(list(set([marker[which] for marker in markers])))
        return sorted_list if up else sorted_list[::-1]
    
    
    def book_best_fit(arr_domain, markers):
        def is_in_dictionary(markers, start, endplus):
            match_starts = [item for item in markers if item[0] == start]
            match_both = [item for item in match_starts if item[1] == endplus]
            return match_both
        
        last_length = [-1]*len(arr_domain)
        for i in range(len(arr_domain)):
            if is_in_dictionary(markers, 0, i+1):
                last_length[i] = i + 1
            if last_length[i] == -1:
                for j in range(i):
                    if last_length[j] != -1 and is_in_dictionary(markers, j+1, i+1):
                        last_length[i] = i - j
                        break
        print("last_length: ", last_length)                
        decompositions = []
        if last_length[-1] != -1:
            idx = len(arr_domain) - 1
            while idx >= 0:
                decompositions.append(is_in_dictionary(markers,idx + 1 - last_length[idx], idx + 1)[0])
                idx -= last_length[idx]
            decompositions = decompositions[::-1]
        return decompositions
    
    
    markers = get_markers(ph.seq, st, lst_entities)
    markers = sift_markers(markers)
#     print("MARKERS TO WORK WITH: ", markers)
    lefts = get_sorted_indexes(markers, 0, up=True)
    rights = get_sorted_indexes(markers, 1, up=False)
    print(lefts)
    print(rights)
    final_decomps = []
    for left in lefts:
        for right in rights:
            subject = ph.w(st)[left:right]
            if not subject or (right - left) <=1:
                continue
            markers = get_markers(ph.seq, " ".join(subject), lst_entities)
            markers = sift_markers(markers)
            decomp = book_best_fit(subject, markers)
            if decomp:
                print("decomp: ", left, right, subject,'\n',decomp, '\n')
                final_decomps.append(decomp)
    #return final_decomps

    best_decomp = None
    max_len = 0
    for decomp in final_decomps:  
        if decomp[-1][1] > max_len:
            max_len = decomp[-1][1]
            best_decomp = decomp
    
    return best_decomp if best_decomp else []
    

def return_max_address4(seq, st):
    lst_final_ents = [['_ADDRESS_', '_SUITE_', '_DIR_'], ['_POBOX_']]
    for lst_entities in lst_final_ents:
        best_fit = get_best_fit_4(seq, st, lst_entities)
        #print("Best_fit: ", best_fit, lst_entities[0])
        if any([fit[3][0] == lst_entities[0] for fit in best_fit]):
            str_rep = " ".join([fit[4] for fit in best_fit])
            return str_rep
    else:
        return []
    
final_decomps = get_best_fit_4(ph.seq, st, ['_ADDRESS_',  '_SUITE_', '_DIR_'])
[print("DECOMP: ", decomp) for decomp in final_decomps]

final = return_max_address4(ph.seq, st)

print("\nAND THE BEST IS\n", final)

[1, 2]
[5, 4]
last_length:  [-1, -1, -1, 4]
decomp:  1 5 ['10', '270', 'webers', 'lane'] 
 [[0, 4, 4, ['_ADDRESS_'], '10 270 webers lane']] 

last_length:  [1, -1, 2]
decomp:  1 4 ['10', '270', 'webers'] 
 [[0, 1, 1, ['_SUITE_'], '10'], [1, 3, 2, ['_ADDRESS_'], '270 webers']] 

last_length:  [-1, 2, 3]
decomp:  2 5 ['270', 'webers', 'lane'] 
 [[0, 3, 3, ['_ADDRESS_'], '270 webers lane']] 

last_length:  [-1, 2]
decomp:  2 4 ['270', 'webers'] 
 [[0, 2, 2, ['_ADDRESS_'], '270 webers']] 

DECOMP:  [0, 4, 4, ['_ADDRESS_'], '10 270 webers lane']
[1, 2]
[5, 4]
last_length:  [-1, -1, -1, 4]
decomp:  1 5 ['10', '270', 'webers', 'lane'] 
 [[0, 4, 4, ['_ADDRESS_'], '10 270 webers lane']] 

last_length:  [1, -1, 2]
decomp:  1 4 ['10', '270', 'webers'] 
 [[0, 1, 1, ['_SUITE_'], '10'], [1, 3, 2, ['_ADDRESS_'], '270 webers']] 

last_length:  [-1, 2, 3]
decomp:  2 5 ['270', 'webers', 'lane'] 
 [[0, 3, 3, ['_ADDRESS_'], '270 webers lane']] 

last_length:  [-1, 2]
decomp:  2 4 ['270', 'webers'] 
 [[0, 

This is where I left off.  The last cell has the new best_fit, which will return the longest substring of continuous subunits containing an address,(Well have to dup it for po boxes if there isn't any.